In [1]:
import altair as alt
import pandas as pd
import glob

In [2]:
source = pd.read_csv("./results/mbarc26/blast/blast_id_heatmap_mbarc26_real16s.vs.real16s.csv")
df = pd.DataFrame(source)

In [3]:
# replaces first name with its initial 
# returns None if string on name indexing error
def firstWordInitial(wholeName: str, delimiter: str = ' '):
    splitName = wholeName.split(delimiter)
    try:
        initial = f"{splitName[0][0]}."
        return wholeName.replace(splitName[0], initial, 1)
    except IndexError:
        return None

In [4]:
adjustedNamesDF = df.copy()

# x-axis
adjustedNamesDF["Subject ID"] = adjustedNamesDF["Subject ID"].apply(firstWordInitial)

In [5]:
# Get alphabetically sorted x-axis by species name before first 'name' initial occurs
sortedRefOneDF = adjustedNamesDF.sort_values("Query ID")
sortedRefOneDF["nameAbbreviation"] = adjustedNamesDF["Query ID"].apply(firstWordInitial)

# order to sort x-axis
explicitXAxisOrder = sortedRefOneDF["nameAbbreviation"].unique()

In [6]:
# explicitNameOrder = [
#   "Clostridium perfringens",
#   "Clostridium thermocellum",
#   "Coraliomargarita akajimensis",
#   "Corynebacterium glutamicum",
#   "Desulfosporosinus acidiphilus",
#   "Desulfosporosinus meridiei",
#   "Desulfotomaculum gibsoniae",
#   "Escherichia coli",
#   "Echinicola vietnamensis",
#   "Fervidobacterium pennivorans",
#   "Frateuria aurantia",
#   "Halovivax ruber",
#   "Hirschia baltica",
#   "Meiothermus silvanus",
#   "Natronobacterium gregoryi",
#   "Natronococcus occultus",
#   "Nocardiopsis dassonvillei",
#   "Olsenella uli",
#   "Pseudomonas stutzeri",
#   "Salmonella bongori",
#   "Salmonella enterica",
#   "Segniliparus rotundus",
#   "Spirochaeta smaragdinae",
#   "Streptococcus pyogenes",
#   "Terriglobus roseus",
#   "Thermobacillus composti"
# ]

In [7]:
base = alt.Chart(
    adjustedNamesDF.copy(),
    title="BLAST ID Percentage"
    ).encode(
    x=alt.X(
        "Subject ID:O",
        scale=alt.Scale(paddingInner=0),
        sort=explicitXAxisOrder,
        axis=alt.Axis(labelAngle=-45),
        title="Reference 16S Query"
    ),
    y=alt.Y(
        "Query ID:O",
        scale=alt.Scale(paddingInner=0),
        # sort=explicitNameOrder,
        title="Reference 16S Subject"
    )
).properties(
    width= 1050,
    height=850
)

heatmap = base.mark_rect().encode(
    color=alt.Color(
        "Percent identity:Q",
        scale=alt.Scale(
            type='log',
            domain=[65,100],
            scheme="blues"),
    )
)

text = base.mark_text(
    baseline="middle",
    fontSize=14
    ).encode(
        text=alt.Text("Percent identity:Q",format=",.0f"),
        color=alt.condition(
            alt.datum["Percent identity"] > 95,
            alt.value("lightgrey"),
            alt.value("black"),
        )
)

heat_plot = heatmap + text

heat_plot.configure_axis(
    titleFontSize = 23,
    labelFontSize = 13
).configure_title(
    fontSize=30
)


/Users/Mary/opt/anaconda3/envs/upset-altair-env/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)